In [1]:
import pandas as pd
import numpy as np
import os
import unicodedata


In [2]:
#Definição de variáveis
pasta_datasets='../Datasets/'
pasta_planilhas='../Planilhas/Combustivel/'

try:
    os.mkdir(pasta_datasets)
    print(f'"A pasta {pasta_datasets} foi criada.') 
except FileExistsError:
    print(f'"A pasta {pasta_datasets} já existe.') 
    


"A pasta ../Datasets/ já existe.


In [3]:
# Função utilizada para remover acentos e demais caracteres especiais de uma string
def ajustar_string(texto):
    texto_ajustado = unicodedata.normalize("NFD", texto)
    texto_ajustado = texto_ajustado.encode("ascii", "ignore")
    texto_ajustado = texto_ajustado.decode("utf-8")
    return texto_ajustado
    

In [4]:
# Cria uma lista com todos os arquivos contidos na pasta_planilhas, os quais serão processados e agrupados
planilhas = os.listdir(pasta_planilhas)

# Cria um dicionário para receber um dataframe por arquivo
d_df={}

# Defina as colunas de interesse
colunas = ['Regiao - Sigla',
           'Estado - Sigla',
           'Municipio',
           #'CNPJ da Revenda',
           'Produto',
           'Data da Coleta',
           'Valor de Venda',
           'Unidade de Medida',
           #'Bandeira'
          ]

# Define os tipos dos dados de cada coluna de interesse
tipos_dados_colunas = {'Regiao - Sigla':str,
                       'Estado - Sigla':str,
                       'Municipio':str,
                       #'CNPJ da Revenda':str,
                       'Produto':str,
                       'Data da Coleta':str,
                       'Valor de Venda':np.float64,
                       'Unidade de Medida':str
                       #'Bandeira':str
                      }

# Percorre todos os arquivos em 'pasta_planilhas'
for planilha in planilhas:
    
    df = pd.read_csv(pasta_planilhas+planilha,
                     sep = ';',
                     decimal=',',
                     low_memory=False, 
                     parse_dates=['Data da Coleta'],
                     dayfirst = True,encoding='utf-8'
                    )
    
    # Como constatou-se que alguns arquivos possuem acentos nos nomes das colunas, optou-se por utilizar a função
    # "ajustar_string" para normalizar estes nomes adequando-os ao padrão do dicionário 'colunas' acima
    for coluna_no_df in df.columns:
        df.rename(columns ={coluna_no_df:ajustar_string(coluna_no_df)}, inplace = True)
    
    # Agrega as colunas de interesse do dataframe lido no dicionário, utilizando como chave o nome do arquivo
    d_df[planilha] = df[colunas]
    
print('Foram inseridos ' + f'{len(d_df)} dataframes no dicionário d_df\n')
print(planilhas)

Foram inseridos 34 dataframes no dicionário d_df

['ca-2005-01.csv', 'ca-2005-02.csv', 'ca-2006-01.csv', 'ca-2006-02.csv', 'ca-2007-01.csv', 'ca-2007-02.csv', 'ca-2008-01.csv', 'ca-2008-02.csv', 'ca-2009-01.csv', 'ca-2009-02.csv', 'ca-2010-01.csv', 'ca-2010-02.csv', 'ca-2011-01.csv', 'ca-2011-02.csv', 'ca-2012-01.csv', 'ca-2012-02.csv', 'ca-2013-01.csv', 'ca-2013-02.csv', 'ca-2014-01.csv', 'ca-2014-02.csv', 'ca-2015-01.csv', 'ca-2015-02.csv', 'ca-2016-01.csv', 'ca-2016-02.csv', 'ca-2017-01.csv', 'ca-2017-02.csv', 'ca-2018-01.csv', 'ca-2018-02.csv', 'ca-2019-01.csv', 'ca-2019-02.csv', 'ca-2020-01.csv', 'ca-2020-02.csv', 'ca-2021-01.csv', 'ca-2021-02.csv']


In [5]:
# Concatena os dataframes num único dataframe
df_combustiveis = pd.concat(d_df,ignore_index=True,sort=False)

display(df_combustiveis)
df_combustiveis.info()


,Regiao - Sigla,Estado - Sigla,Municipio,Produto,Data da Coleta,Valor de Venda,Unidade de Medida
0,SE,SP,GUARULHOS,GASOLINA,2005-01-04,2.257,R$ / litro
1,SE,SP,GUARULHOS,ETANOL,2005-01-04,1.449,R$ / litro
2,SE,SP,GUARULHOS,DIESEL,2005-01-04,1.579,R$ / litro
3,SE,SP,SOROCABA,GASOLINA,2005-01-04,2.210,R$ / litro
4,SE,SP,SOROCABA,ETANOL,2005-01-04,1.230,R$ / litro
...,...,...,...,...,...,...,...
20110009,NE,BA,JEQUIE,GASOLINA,2021-12-31,6.799,R$ / litro
20110010,SE,MG,ITUIUTABA,ETANOL,2021-12-31,4.740,R$ / litro
20110011,SE,MG,ITUIUTABA,DIESEL S10,2021-12-31,5.448,R$ / litro
20110012,SE,MG,ITUIUTABA,GASOLINA ADITIVADA,2021-12-31,6.580,R$ / litro


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20110014 entries, 0 to 20110013
Data columns (total 7 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Regiao - Sigla     object        
 1   Estado - Sigla     object        
 2   Municipio          object        
 3   Produto            object        
 4   Data da Coleta     datetime64[ns]
 5   Valor de Venda     float64       
 6   Unidade de Medida  object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.0+ GB


In [6]:
# Simplifica o nome de algumas colunas
colunas_renomeadas = {'Data da Coleta': 'Data',
                      'Estado - Sigla' :'Estado',
                      'Regiao - Sigla' : 'Regiao',
                      'Valor de Venda' : 'ValorVenda',
                      'Unidade de Medida' : 'UnMed'
                     }

df_combustiveis.rename(columns=colunas_renomeadas, inplace = True)

print(df_combustiveis.columns)
df_combustiveis.info()

Index(['Regiao', 'Estado', 'Municipio', 'Produto', 'Data', 'ValorVenda',
       'UnMed'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20110014 entries, 0 to 20110013
Data columns (total 7 columns):
 #   Column      Dtype         
---  ------      -----         
 0   Regiao      object        
 1   Estado      object        
 2   Municipio   object        
 3   Produto     object        
 4   Data        datetime64[ns]
 5   ValorVenda  float64       
 6   UnMed       object        
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.0+ GB


In [7]:
# Verifica se há algum valor nulo no dataframe 'df_combustiveis'

df_combustiveis.isnull().values.any()

True

In [8]:
# Verifica os tipos de combustíveis disponíveis na coluna Produto.
df_combustiveis['Produto'].unique()

array(['GASOLINA', 'ETANOL', 'DIESEL', 'GNV', 'DIESEL S50', 'DIESEL S10',
       'ETANO', 'GASOLINA ADITIVADA'], dtype=object)

In [9]:
# Como constatou a existência do produto 'ETANO', verificou-se se era um erro de tipo, e se 
# este dado deveria ser tratado
display(df_combustiveis[df_combustiveis.Produto == 'ETANO'])

,Regiao,Estado,Municipio,Produto,Data,ValorVenda,UnMed
13020846,S,RS,PELOTAS,ETANO,NaT,NaN,NaN


In [10]:
# Neste estudo serão analisados apenas os combustíveis 'GASOLINA' e 'ETANOL'. 
# Não serão considerados assim a 'GASOLINA ADITIVADA', 'DIESEL' e suas variantes e também 'GNV'
# Como o dado do produto 'ETANO' é um erro, conforme acima, ele também será ignorado.
aceitar = ['GASOLINA','ETANOL']

df_combustiveis.drop(df_combustiveis[~df_combustiveis.Produto.isin(aceitar)].index, inplace=True)

display(df_combustiveis)

,Regiao,Estado,Municipio,Produto,Data,ValorVenda,UnMed
0,SE,SP,GUARULHOS,GASOLINA,2005-01-04,2.257,R$ / litro
1,SE,SP,GUARULHOS,ETANOL,2005-01-04,1.449,R$ / litro
3,SE,SP,SOROCABA,GASOLINA,2005-01-04,2.210,R$ / litro
4,SE,SP,SOROCABA,ETANOL,2005-01-04,1.230,R$ / litro
6,CO,DF,BRASILIA,GASOLINA,2005-01-03,2.150,R$ / litro
...,...,...,...,...,...,...,...
20110006,SE,RJ,CAMPOS DOS GOYTACAZES,GASOLINA,2021-12-31,7.090,R$ / litro
20110007,NE,BA,JEQUIE,ETANOL,2021-12-31,5.859,R$ / litro
20110009,NE,BA,JEQUIE,GASOLINA,2021-12-31,6.799,R$ / litro
20110010,SE,MG,ITUIUTABA,ETANOL,2021-12-31,4.740,R$ / litro


In [11]:
# Verifica se há mais de uma unidade de medida
df_combustiveis['UnMed'].unique()

array(['R$ / litro'], dtype=object)

In [12]:
# Como a unidade de medida é sempre a mesma, a coluna "UnMed" não será mais necessária
df_combustiveis.drop(columns='UnMed', inplace=True)

In [13]:
df_combustiveis.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12900965 entries, 0 to 20110013
Data columns (total 6 columns):
 #   Column      Dtype         
---  ------      -----         
 0   Regiao      object        
 1   Estado      object        
 2   Municipio   object        
 3   Produto     object        
 4   Data        datetime64[ns]
 5   ValorVenda  float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 689.0+ MB


In [14]:
# Verifica se há algum valor nulo no dataframe 'df_combustiveis'
df_combustiveis.isnull().values.any()

False

In [15]:
# Cria um novo índice com a coluna 'Data' 
df_combustiveis.set_index(['Data'], inplace=True)

display(df_combustiveis)

,Regiao,Estado,Municipio,Produto,ValorVenda
Data,,,,,
2005-01-04,SE,SP,GUARULHOS,GASOLINA,2.257
2005-01-04,SE,SP,GUARULHOS,ETANOL,1.449
2005-01-04,SE,SP,SOROCABA,GASOLINA,2.210
2005-01-04,SE,SP,SOROCABA,ETANOL,1.230
2005-01-03,CO,DF,BRASILIA,GASOLINA,2.150
...,...,...,...,...,...
2021-12-31,SE,RJ,CAMPOS DOS GOYTACAZES,GASOLINA,7.090
2021-12-31,NE,BA,JEQUIE,ETANOL,5.859
2021-12-31,NE,BA,JEQUIE,GASOLINA,6.799


In [16]:
# Verifica a menor e a maior data no dataframe 'df_combustiveis'
menor_data = df_combustiveis.index.min()
maior_data = df_combustiveis.index.max()

print('Menor data no dataframe df_combustiveis: ' + str(menor_data))
print('Maior data no dataframe df_combustiveis: ' + str(maior_data))

Menor data no dataframe df_combustiveis: 2005-01-03 00:00:00
Maior data no dataframe df_combustiveis: 2021-12-31 00:00:00


In [17]:
# Exporta o dataset para um arquivo CSV
df_combustiveis.to_csv(pasta_datasets+'combustiveis.csv', sep = ';',index=True)

